# Python Live Project - Thinking about how to Manipulate Data

In this liveProject you were just hired by the NBA to gather basic data on every NBA all-star basketball games that has happened until 2017. The all-star games within that period pitted fan-favorite players from the Eastern Conference vs fan-favorite players from the Western Conference. The question you are tasked with answering is:

 - What is the maximum and minimum score difference between all games?
 - If a city hosted a game more than once, what was the average score for the Eastern Conference team and the Western Conference team?


# Milestone 1

In [219]:
"""
Read a table from a website with the data you are interested in. 
The data will contain some rows that do not contain useful data – remove them. 
The outcome is a file that can read the table from the link, 
save it as a DataFrame, and print the DataFrame to the console. 
The printed DataFrame should not contain any rows with NA in them.
"""


#importing data and reading it from the site
import pandas as pd 
data = pd.read_html("https://en.wikipedia.org/wiki/NBA_All-Star_Game")[2]
#dropping the NA values and what we don't need
data = data.dropna()
data


,Year,Result,Host arena,Host city,Game MVP
0,1951,"East 111, West 94",Boston Garden,"Boston, Massachusetts","Ed Macauley, Boston Celtics"
1,1952,"East 108, West 91",Boston Garden (2),"Boston, Massachusetts (2)","Paul Arizin, Philadelphia Warriors"
2,1953,"West 79, East 75",Allen County War Memorial Coliseum,"Fort Wayne, Indiana","George Mikan, Minneapolis Lakers"
3,1954,"East 98, West 93 (OT)",Madison Square Garden III**,"New York City, New York","Bob Cousy, Boston Celtics"
4,1955,"East 100, West 91",Madison Square Garden III** (2),"New York City, New York (2)","Bill Sharman, Boston Celtics"
...,...,...,...,...,...
66,2017,"West 192, East 182",Smoothie King Center (3),"New Orleans, Louisiana (3)","Anthony Davis, New Orleans Pelicans"
67,2018[5],"Team LeBron 148, Team Stephen 145",Staples Center (3),"Los Angeles, California (4)","LeBron James (3), Cleveland Cavaliers"
68,2019,"Team LeBron 178, Team Giannis 164",Spectrum Center,"Charlotte, North Carolina (2)","Kevin Durant (2), Golden State Warriors"
69,2020,"Team LeBron 157, Team Giannis 155‡",United Center,"Chicago, Illinois (3)","Kawhi Leonard, Los Angeles Clippers"


# Milestone 2

In [220]:
"""
 remove columns that we don’t care about and make new columns with information you care about. 
 All this is done on a row-by-row basis, thus keeping all the original rows. 
 The outcome is code that prints a DataFrame containing 3 columns: 
 Eastern Conference score, Western conference score, and the Host city. The column index is the names of the columns. 
 
 """
#Cleaning the data
data = data.drop(columns=["Game MVP", "Host arena"])

#This was a weird of removing some chars I was having an issue with - sorry lol
data["Host city"] = [x.replace('(2)', '').replace('(3)','').replace('(4)','').replace('†','').replace('.[17]','')\
                     .replace('#','').replace('(5)','').replace('[5]','').replace('*','')for x in data["Host city"]]
data["Host city"] = [x.split(',')[1].lstrip().split(' ') for x in data["Host city"]]
data["Host city"] = [''.join(x) for x in data["Host city"]]

import regex as re
data["New results"] = [re.findall('[0-9]+', x) for x in data["Result"]]
#break up new results into two colums one for each int 
data["East team score"] = [x[0] for x in data["New results"]]
data["West team score"] = [x[-1:] for x in data["New results"]]
data["West team score"] = [''.join(x) for x in data["West team score"]]
data.drop(columns=['Result', 'New results'])
data.set_index('Year')

,Result,Host city,New results,East team score,West team score
Year,,,,,
1951,"East 111, West 94",Massachusetts,"[111, 94]",111,94
1952,"East 108, West 91",Massachusetts,"[108, 91]",108,91
1953,"West 79, East 75",Indiana,"[79, 75]",79,75
1954,"East 98, West 93 (OT)",NewYork,"[98, 93]",98,93
1955,"East 100, West 91",NewYork,"[100, 91]",100,91
...,...,...,...,...,...
2017,"West 192, East 182",Louisiana,"[192, 182]",192,182
2018[5],"Team LeBron 148, Team Stephen 145",California,"[148, 145]",148,145
2019,"Team LeBron 178, Team Giannis 164",NorthCarolina,"[178, 164]",178,164


# Milestone 3 

In [221]:
"""
Report counts for the score differences. 
Determine which final score difference between the two teams occurred most often. 
The end outcome is code that prints a DataFrame whose index is the difference, containing 1 column: 
the count of the difference. 
"""
#removing the results col and puting getting the difference for each of the score
data = data.drop(columns=["Result", "New results"])
data["East team score"] = [int(x) for x in data["East team score"]]
data["West team score"] = [int(x) for x in data["West team score"]]
data["Score diff"] = data["East team score"]- data["West team score"]
data

,Year,Host city,East team score,West team score,Score diff
0,1951,Massachusetts,111,94,17
1,1952,Massachusetts,108,91,17
2,1953,Indiana,79,75,4
3,1954,NewYork,98,93,5
4,1955,NewYork,100,91,9
...,...,...,...,...,...
66,2017,Louisiana,192,182,10
67,2018[5],California,148,145,3
68,2019,NorthCarolina,178,164,14
69,2020,Illinois,157,155,2


In [205]:
#grouping the columns byt the score difference
groupDiff = data.groupby("Score diff")
differenceDF = pd.DataFrame(data = groupDiff.size(), index = groupDiff.size())
#setting the index name to be Score diff so it is clearer
differenceDF.index.name = 'Score diff'
differenceDF.sort_values(0, ascending=False)

,0
Score diff,
5,7
5,7
2,6
2,6
2,6
2,6
1,4
1,4
1,4


# Milestone 4

In [196]:
"""
Group the DataFrame by the Host city. Use the mean as the function to apply on columns 
where there is more than one value to group together. 

The index should now be the Host city instead of the Year.

"""
#group by host city and getting the mean values 
hgroup = data.groupby("Host city")
hgroupDF = pd.DataFrame(hgroup.mean())
#counting the values of the host cities and adding them to a new column called Count showing values above 1
hgroupDF["Count"] = data["Host city"].value_counts()
hgroupDF.drop(hgroupDF.loc[2 > hgroupDF['Count']].index, inplace=True)
hgroupDF.drop(columns='Score diff').sort_values("Count")

"""
If a city hosted a game more than once, 
what was the average score for the Eastern Conference team and the Western Conference team?
"""

,East team score,West team score,Count
Host city,,,
Maryland,133.500000,124.000000,2
NorthCarolina,147.000000,139.000000,2
Michigan,129.000000,118.500000,2
Indiana,109.500000,102.000000,2
Washington,144.000000,136.000000,2
Colorado,139.500000,130.000000,2
Illinois,133.000000,124.000000,3
Florida,145.000000,125.000000,3
Louisiana,163.000000,155.000000,3
